<a href="https://colab.research.google.com/github/shubhagrawal8999/self-aprtice-agents-/blob/main/Agentic_AI_week1_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# * Here we are going to make our owm advanced chat bot *

In [ ]:
%pip install openai

*_______________________________________*

**Here we are going to used pushover **


Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your .env file:

PUSHOVER_USER=put the key that's on the top right of your Pushover home screen and probably starts with a u
PUSHOVER_TOKEN=put the key when you click into your new application called Agents (or whatever) and probably starts with an a

Remember to save your .env file, and run load_dotenv(override=True) after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [1]:
from openai import OpenAI
import json
import os
import requests
import gradio as gr

In [4]:
from google.colab import userdata
api_key =userdata.get("openai_api_key")
openai = OpenAI(api_key=api_key)

In [5]:
from google.colab import userdata
pushover_user =userdata.get('PUSHOVER_USER')
pushover_token =userdata.get('PUSHOVER_TOKEN')
# This is the url of the website
pushover_url = "https://api.pushover.net/1/messages.json"

# checking if pushover api is sucessfull add or not
if pushover_user and pushover_token:
  print("Pushover API is working")
else:
  print("pushover API is not working")


Pushover API is working


In [6]:
def push(message):
    print(f"push:{message}")
    payload ={"user":pushover_user,"token":pushover_token,"message":message}
    requests.post(pushover_url,data=payload)

In [7]:
push("Hey!!")


push:Hey!!


In [8]:
def record_user_details(email,name="Name not provided",notes="notProvided"):
  push(f"Recording interset from {name} with email {email} and notes {notes}")
  return {"recording":"ok"}


In [9]:
def record_unknown_question(question):
  push(f"Recording {question} asked that I culdnot answer")
  return {"recorded":"ok"}


In [10]:
record_user_details_json ={
    "name":"record_user_details",
    "description":"Use this tool to record that a user is interseted in being in touch and provided and email address",
    "parameters":{
        "type":"object",
        "properties":{
            "email":{
                "type":"string",
                "description":"The email address of this user"
            },
            "name":{
                "type":"string",
                "description":"The user name if they proivded"
            },
             "notes":{
                "type":"string",
                "description":"Any additional information about the conversation that's worth recording to givecontext"

            }
        },

            "required":["email"],
            "additionalProperties":False
        }
}

In [11]:
record_unknown_question_json ={
    "name":"record_unknown_question",
    "description":"Always use this tool to record any question that couldnot be answered as you donot known the answer ",
    "parameters":{
        "type":"object",
        "properties":{
            "question":{
                "type":"string",
                "description":"The question that cannot be answered"
            },

        },
        "required":["question"],
        "additionalProperties":False
    }
}

In [13]:
tools= [{"type":"function","function":record_user_details_json},
        {"type":"function","function":record_unknown_question_json}]


In [14]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interseted in being in touch and provided and email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': 'The user name if they proivded'},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to givecontext"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Always use this tool to record any question that couldnot be answered as you donot known the answer ',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'The question that cannot be answered'}},
    'required': ['question'],


# This function can take a list of tool calls, and run them. this is the IF statement!!

In [15]:
def handle_tool_calls(tool_calls):
  results =[]
  for tool_call in tool_calls:
    tool_name= tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    print(f"tool call:{tool_name}",flush=True)

    # The BIG IF STATEMENT!!!

    if tool_name == "record_user_details":
      result = record_user_details(**arguments)
    elif tool_name == "record_unknown_question":
      result = record_unknown_question(**arguments)


    results.append({"role":"tool","content":json.dumps(result),"tool_call_id":tool_call.id})

  return results








In [16]:
globals()["record_unknown_question"]("This is a really a hard question")


push:Recording This is a really a hard question asked that I culdnot answer


{'recorded': 'ok'}

In [28]:
def handle_tool_calls(tool_calls):
  results = []
  for tool_call in tool_calls:
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    print(f"Tools called :{tool_name}",flush=True)
    tool = globals().get(tool_name)
    result = tool(**arguments) if tool else{}
    results.append({"role":"tool","content":json.dumps(result),"tool_call_id":tool_call.id})
  return results

To import your LinkedIn CSV file, you first need to upload it to your Colab environment. You can do this by clicking the 'Files' icon on the left sidebar, then clicking the 'Upload to session storage' icon (a document with an arrow pointing up) and selecting your CSV file. Once uploaded, you can use the following Python code to read it into a pandas DataFrame:

In [18]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
linkedin = pd.read_csv('/content/drive/MyDrive/Profile.csv')

Mounted at /content/drive


In [20]:
linkedin

,First Name,Last Name,Maiden Name,Address,Birth Date,Headline,Summary,Industry,Zip Code,Geo Location,Twitter Handles,Websites,Instant Messengers
0,Shubh,agrawal,NaN,NaN,Mar 31,Student at SHARAD PAWAR INTERNATIONAL SCHOOL &...,"currently i am learning coding,ai automation a...",IT Services and IT Consulting,422003,"Nandurbar, Maharashtra, India",NaN,[PERSONAL:www.zubhai.com],NaN


In [21]:
name = "shubh Agrwal"

In [23]:
system_prompt = f"You are acting as {name},You are asnwering questions on {name}'s website .\
particularly questions related to {name}'s career,background ,skillsand experience. \
You are given a summary of {name}'s background and linkedin profile which you can use to answer question .\
BE prfessional and engaging . as if talking to a potential_client or future employer who came across the website .\
If you donot know the answer to any question ,use to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool."

system_prompt += f"\n\n##LinkedIn profile:\n{linkedin}\n\n"
system_prompt += f"with this context, pleases chat with the user, always staying om character as {name}."


In [26]:
def chat(message,history):
  messages = [{"role":"system","content":system_prompt}] + history +[{"role":"user","content":message}]
  done =False
  while not done:

    #This is the call to the LLm - see that we pass in the tools json

    response = openai.chat.completions.create(model="gpt-5-nano",messages = messages ,tools =tools)

    finish_reason = response.choices[0].finish_reason


      # IF the LLm eants to call a tool ,we do that!

    if finish_reason =="tool_calls":
        message = response.choices[0].message
        tool_calls =message.tool_calls
        results = handle_tool_calls(tool_calls)
        messages.extend(results)

    else:
        done =True

  return response.choices[0].message.content



In [27]:
gr.ChatInterface(chat,type="messages").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6389163af51963312.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
